<a href="https://colab.research.google.com/github/LADISLAOCOSTILLO/ARCHIVOSGIT/blob/main/Pr%C3%A1ctica_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL**

Spark SQL es un módulo de Apache Spark que facilita el procesamiento de datos estructurados mediante SQL. Permite trabajar con DataFrames y Dataset, ofreciendo una forma eficiente de realizar análisis de datos utilizando consultas SQL estándar o las APIs de Spark en lenguajes como Python.

# 1. Instalación de PySpark en Google Colab

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Consultar sitio para obtener el enlace de descarga más reciente https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [5,290 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

In [ ]:
spark

# 2. Leer Data COVID-19

Se utiliza un conjunto de datos disponibles públicamente en formato CSV. Rastreador de COVID-19 es un proyecto de análisis de datos basado en Python que rastrea los casos, muertes y tendencias de COVID-19 en países seleccionados utilizando datos reales de Our World in Data (OWID).

In [2]:
import requests
path = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'owid-covid-data.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

df = spark.read.csv('/content/'+csv_file_name, header=True, inferSchema=True)

El módulo SQL es muy accesible para interactuar con los datos mientras se sigue usando Spark. Es básicamente la misma sintaxis SQL.

In [3]:
#Creación de una tabla a partir del marco de datos
df.createOrReplaceTempView("covid_data") #Vista temporal
# df.saveAsTable("covid_data") #Guardar como una tabla
# df.write.mode("overwrite").saveAsTable("covid_data") #Guardar como tabla y sobrescribir la tabla si existe

In [4]:
groupDF = spark.sql("SELECT location, count(*) from covid_data group by location")
groupDF.show()

+--------------------+--------+
|            location|count(1)|
+--------------------+--------+
|                Chad|    1674|
|            Anguilla|    1674|
|            Kiribati|    1674|
|              Guyana|    1674|
|             Eritrea|    1674|
|              Jersey|    1674|
|            Djibouti|    1674|
|                Fiji|    1674|
|                Iraq|    1674|
|              Europe|    1684|
|             Germany|    1674|
|             Comoros|    1674|
|         Afghanistan|    1674|
|            Cambodia|    1674|
|High-income count...|    3026|
|              Jordan|    1674|
|              France|    1674|
|              Greece|    1674|
|              Kosovo|    1674|
|              Africa|    1674|
+--------------------+--------+
only showing top 20 rows



# 3. En este caso revisamos el siguiente conjunto de datos que viene con la sesión de Google Colab

In [5]:
df = spark.read.csv("/content/sample_data/california_housing_train.csv", header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [7]:
#Imprimir N filas
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
+---------+--------+----

In [8]:
df.count()

17000

In [9]:
df.select("housing_median_age","total_rooms").show(5)

+------------------+-----------+
|housing_median_age|total_rooms|
+------------------+-----------+
|              15.0|     5612.0|
|              19.0|     7650.0|
|              17.0|      720.0|
|              14.0|     1501.0|
|              20.0|     1454.0|
+------------------+-----------+
only showing top 5 rows



In [10]:
df.describe().show()

+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|          latitude|housing_median_age|      total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|
+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              17000|             17000|             17000|            17000|            17000|             17000|            17000|             17000|             17000|
|   mean|-119.56210823529375|  35.6252247058827| 28.58935294117647|2643.664411764706|539.4108235294118|1429.5739411764705|501.2219411764706| 3.883578100000021|207300.91235294117|
| stddev| 2.0051664084260357|2.1373397946570867|12.586936981660406|2179.947071452777|421.4994515798648| 1

In [11]:
df.select('total_rooms').distinct().show()

+-----------+
|total_rooms|
+-----------+
|      934.0|
|     3980.0|
|     4142.0|
|      596.0|
|     1761.0|
|     5983.0|
|     2815.0|
|     6433.0|
|      299.0|
|     2734.0|
|      769.0|
|     1051.0|
|     7554.0|
|     4066.0|
|     2862.0|
|     3597.0|
|      692.0|
|      720.0|
|     1765.0|
|     2523.0|
+-----------+
only showing top 20 rows



In [12]:
from pyspark.sql import functions as F
test = df.groupBy('total_rooms').agg(F.sum('housing_median_age'))

In [13]:
test.toPandas()

,total_rooms,sum(housing_median_age)
0,934.0,135.0
1,3980.0,25.0
2,4142.0,37.0
3,596.0,25.0
4,1761.0,154.0
...,...,...
5528,3620.0,18.0
5529,947.0,62.0
5530,710.0,52.0
5531,91.0,43.0


from matplotlib import pyplot as plt
_df_0['total_rooms'].plot(kind='hist', bins=20, title='total_rooms')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['sum(housing_median_age)'].plot(kind='hist', bins=20, title='sum(housing_median_age)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='total_rooms', y='sum(housing_median_age)', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['total_rooms'].plot(kind='line', figsize=(8, 4), title='total_rooms')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_4['sum(housing_median_age)'].plot(kind='line', figsize=(8, 4), title='sum(housing_median_age)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [14]:
#Contar y eliminar valores faltantes

df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+



Creación de un marco de datos de prueba de Spark

In [15]:
data = [
        ('John','Smith',1),
        ('Jane','Smith',2),
        ('Jonas','Smith',3),
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

IndexError: list index out of range

In [ ]:
df

# Spark Tips
Esta es una colección de fragmentos de código para tareas comunes o complejas.

# Marco de datos de Pandas a marco de datos de Spark

In [16]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randint(100,size=(1000, 3)),columns=['A','B','C'])
spark_df = spark.createDataFrame(df)
spark_df.show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
| 44|  6| 71|
| 86| 20| 39|
|  9| 26| 82|
| 12| 24| 96|
| 14| 73| 46|
|  0| 33|  4|
|  2| 60| 70|
| 97| 63| 48|
| 75| 19| 87|
| 24| 78| 47|
| 25| 43| 12|
|  1| 34| 72|
| 58| 78| 96|
| 38| 47| 97|
| 74|  3| 40|
| 88| 76| 75|
| 76| 81| 34|
| 62| 26| 61|
| 72| 14| 17|
|  4| 55| 70|
+---+---+---+
only showing top 20 rows



In [ ]:
#Convertir columnas de objeto en un marco de datos de pandas en una cadena
#for i in df.select_dtypes(include='object').columns.tolist():
#	df[i] = df[i].astype(str)

#Reemplazar nan y "None" en pandas dataframe a null en el marco de datos de spark
#spark_df = spark.createDataFrame(df).replace('None', None).replace(float('nan'), None)

# Funciones ventana SQL
En SQL, las funciones ventana (o funciones analíticas) son funciones que realizan cálculos en un conjunto de filas relacionadas con la fila actual, sin colapsar las filas en una sola fila de salida (a diferencia de las funciones de agregación regulares con GROUP BY).

In [17]:
data = [
        (1,'2021-01-01 10:00:00'),
        (1,'2021-01-01 11:00:00'),
        (1,'2021-01-01 12:00:00'),
        (2,'2021-01-01 12:00:00'),
        (2,'2021-01-01 13:00:00'),
        (2,'2021-01-01 14:00:00'),
]

columns = ["id","datetime"]
df = spark.createDataFrame(data=data, schema = columns)
df.createOrReplaceTempView("window_test")
df.show()

+---+-------------------+
| id|           datetime|
+---+-------------------+
|  1|2021-01-01 10:00:00|
|  1|2021-01-01 11:00:00|
|  1|2021-01-01 12:00:00|
|  2|2021-01-01 12:00:00|
|  2|2021-01-01 13:00:00|
|  2|2021-01-01 14:00:00|
+---+-------------------+



In [18]:
#Seleccionando min y max por un Grupo específico
spark.sql('''
Select
  id,

  max(datetime) OVER (Partition BY id ORDER BY datetime) as max_date,
  min(datetime) OVER (Partition BY id ORDER BY datetime) as min_date,

  ROW_NUMBER() OVER (Partition BY id ORDER BY datetime) as row_number

  FROM window_test

''').show()

+---+-------------------+-------------------+----------+
| id|           max_date|           min_date|row_number|
+---+-------------------+-------------------+----------+
|  1|2021-01-01 10:00:00|2021-01-01 10:00:00|         1|
|  1|2021-01-01 11:00:00|2021-01-01 10:00:00|         2|
|  1|2021-01-01 12:00:00|2021-01-01 10:00:00|         3|
|  2|2021-01-01 12:00:00|2021-01-01 12:00:00|         1|
|  2|2021-01-01 13:00:00|2021-01-01 12:00:00|         2|
|  2|2021-01-01 14:00:00|2021-01-01 12:00:00|         3|
+---+-------------------+-------------------+----------+



In [19]:
# Seleccionar el número de fila o el rango de orden para cada fila dentro de una agrupación específica.
# Esto es ideal para subclasificaciones en una tabla.

spark.sql('''
Select
  id,
  datetime,

  ROW_NUMBER() OVER (Partition BY id ORDER BY datetime) as row_number

  FROM window_test

''').show()

+---+-------------------+----------+
| id|           datetime|row_number|
+---+-------------------+----------+
|  1|2021-01-01 10:00:00|         1|
|  1|2021-01-01 11:00:00|         2|
|  1|2021-01-01 12:00:00|         3|
|  2|2021-01-01 12:00:00|         1|
|  2|2021-01-01 13:00:00|         2|
|  2|2021-01-01 14:00:00|         3|
+---+-------------------+----------+



# De-duplicated data: Desduplicar datos devolviendo la fila actualizada más recientemente mediante una función de ventana

In [20]:
data = [
        (1,'2021-01-01',100,'A'),
        (1,'2021-01-31',105,'A'),
        (2,'2021-02-04',160,'B'),
        (2,'2021-02-07',145,'B'),
]

columns = ["id","date","score","type"]
df = spark.createDataFrame(data=data, schema = columns)
df.createOrReplaceTempView("window_test")
df.show()

+---+----------+-----+----+
| id|      date|score|type|
+---+----------+-----+----+
|  1|2021-01-01|  100|   A|
|  1|2021-01-31|  105|   A|
|  2|2021-02-04|  160|   B|
|  2|2021-02-07|  145|   B|
+---+----------+-----+----+



In [21]:
df2 = spark.sql("""
WITH T AS (
  SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY id ORDER BY date DESC) AS version_number
  FROM window_test
)

SELECT * FROM T WHERE version_number = 1;

""")

df2.show()

+---+----------+-----+----+--------------+
| id|      date|score|type|version_number|
+---+----------+-----+----+--------------+
|  1|2021-01-31|  105|   A|             1|
|  2|2021-02-07|  145|   B|             1|
+---+----------+-----+----+--------------+



In [22]:
spark.sql("""
  SELECT
  *,
  SUM(score) OVER (PARTITION by type ORDER BY date) as score_cumulative
  FROM window_test

""").show()

+---+----------+-----+----+----------------+
| id|      date|score|type|score_cumulative|
+---+----------+-----+----+----------------+
|  1|2021-01-01|  100|   A|             100|
|  1|2021-01-31|  105|   A|             205|
|  2|2021-02-04|  160|   B|             160|
|  2|2021-02-07|  145|   B|             305|
+---+----------+-----+----+----------------+



## Limitar el número de resultados por función de ventana de grupo

In [24]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
np.hstack((
    np.random.randint(1,5,size=(100000, 1)),
    np.random.randint(100,size=(100000, 1))
))
, columns=['company_id', 'number'])

dff = spark.createDataFrame(df)
dff.createOrReplaceTempView("window_test_limits")

In [ ]:
spark.sql("""
WITH T AS (
  SELECT
    company_id,
    number,
    ROW_NUMBER() OVER (PARTITION BY company_id ORDER BY number) AS row_number
  FROM window_test_limits
    )

SELECT * FROM T WHERE row_number <= 100

""").show()

# Calcular una media móvil de 7 días

In [25]:
df = pd.DataFrame(pd.date_range('1/1/2022','1/31/2022',freq='D'), columns=['date'])
import random
df['company_id'] = 1
df['number'] = df.apply(lambda x: random.randint(0,100), axis = 1)

dff = spark.createDataFrame(df)
dff.createOrReplaceTempView("window_data")

dff.show()

+-------------------+----------+------+
|               date|company_id|number|
+-------------------+----------+------+
|2022-01-01 00:00:00|         1|    36|
|2022-01-02 00:00:00|         1|    26|
|2022-01-03 00:00:00|         1|    50|
|2022-01-04 00:00:00|         1|    90|
|2022-01-05 00:00:00|         1|    46|
|2022-01-06 00:00:00|         1|    27|
|2022-01-07 00:00:00|         1|    92|
|2022-01-08 00:00:00|         1|    15|
|2022-01-09 00:00:00|         1|     5|
|2022-01-10 00:00:00|         1|    93|
|2022-01-11 00:00:00|         1|    96|
|2022-01-12 00:00:00|         1|     5|
|2022-01-13 00:00:00|         1|   100|
|2022-01-14 00:00:00|         1|    65|
|2022-01-15 00:00:00|         1|    83|
|2022-01-16 00:00:00|         1|    16|
|2022-01-17 00:00:00|         1|    76|
|2022-01-18 00:00:00|         1|    24|
|2022-01-19 00:00:00|         1|    78|
|2022-01-20 00:00:00|         1|    43|
+-------------------+----------+------+
only showing top 20 rows



In [26]:
spark.sql("""
SELECT
  date,
  company_id,
  number,
  AVG(number) OVER (PARTITION BY company_id ORDER BY date ASC RANGE BETWEEN INTERVAL 6 DAYS PRECEDING AND CURRENT ROW) as last_7_day_avg
FROM window_data
""").show()

+-------------------+----------+------+------------------+
|               date|company_id|number|    last_7_day_avg|
+-------------------+----------+------+------------------+
|2022-01-01 00:00:00|         1|    36|              36.0|
|2022-01-02 00:00:00|         1|    26|              31.0|
|2022-01-03 00:00:00|         1|    50|37.333333333333336|
|2022-01-04 00:00:00|         1|    90|              50.5|
|2022-01-05 00:00:00|         1|    46|              49.6|
|2022-01-06 00:00:00|         1|    27|45.833333333333336|
|2022-01-07 00:00:00|         1|    92| 52.42857142857143|
|2022-01-08 00:00:00|         1|    15| 49.42857142857143|
|2022-01-09 00:00:00|         1|     5| 46.42857142857143|
|2022-01-10 00:00:00|         1|    93| 52.57142857142857|
|2022-01-11 00:00:00|         1|    96| 53.42857142857143|
|2022-01-12 00:00:00|         1|     5| 47.57142857142857|
|2022-01-13 00:00:00|         1|   100|              58.0|
|2022-01-14 00:00:00|         1|    65|54.14285714285714

Usuarios activos mensualmente

In [27]:
import pandas as pd
df = pd.DataFrame(pd.date_range('1/1/2022','1/31/2022',freq='D'), columns=['login_date'])
import random
df['company_id'] = 1
df['user_id'] = df.apply(lambda x: random.randint(0,3), axis = 1)

dff = spark.createDataFrame(df)
dff.createOrReplaceTempView("users_data")

dff.show()

+-------------------+----------+-------+
|         login_date|company_id|user_id|
+-------------------+----------+-------+
|2022-01-01 00:00:00|         1|      2|
|2022-01-02 00:00:00|         1|      2|
|2022-01-03 00:00:00|         1|      2|
|2022-01-04 00:00:00|         1|      2|
|2022-01-05 00:00:00|         1|      3|
|2022-01-06 00:00:00|         1|      1|
|2022-01-07 00:00:00|         1|      1|
|2022-01-08 00:00:00|         1|      0|
|2022-01-09 00:00:00|         1|      1|
|2022-01-10 00:00:00|         1|      0|
|2022-01-11 00:00:00|         1|      2|
|2022-01-12 00:00:00|         1|      1|
|2022-01-13 00:00:00|         1|      0|
|2022-01-14 00:00:00|         1|      1|
|2022-01-15 00:00:00|         1|      0|
|2022-01-16 00:00:00|         1|      2|
|2022-01-17 00:00:00|         1|      3|
|2022-01-18 00:00:00|         1|      3|
|2022-01-19 00:00:00|         1|      0|
|2022-01-20 00:00:00|         1|      3|
+-------------------+----------+-------+
only showing top

In [28]:
#Revisar esta transformación
spark.sql("""
SELECT
  login_date,
  COUNT(user_id) OVER (PARTITION BY login_date ORDER BY login_date ASC RANGE BETWEEN INTERVAL 30 DAYS PRECEDING AND CURRENT ROW) AS monthly_active_users
  FROM users_data
""").show()

+-------------------+--------------------+
|         login_date|monthly_active_users|
+-------------------+--------------------+
|2022-01-01 00:00:00|                   1|
|2022-01-02 00:00:00|                   1|
|2022-01-03 00:00:00|                   1|
|2022-01-04 00:00:00|                   1|
|2022-01-05 00:00:00|                   1|
|2022-01-06 00:00:00|                   1|
|2022-01-07 00:00:00|                   1|
|2022-01-08 00:00:00|                   1|
|2022-01-09 00:00:00|                   1|
|2022-01-10 00:00:00|                   1|
|2022-01-11 00:00:00|                   1|
|2022-01-12 00:00:00|                   1|
|2022-01-13 00:00:00|                   1|
|2022-01-14 00:00:00|                   1|
|2022-01-15 00:00:00|                   1|
|2022-01-16 00:00:00|                   1|
|2022-01-17 00:00:00|                   1|
|2022-01-18 00:00:00|                   1|
|2022-01-19 00:00:00|                   1|
|2022-01-20 00:00:00|                   1|
+----------

# Encontrar la diferencia de tiempo entre filas relacionadas usando una función de ventana

In [29]:
data = [
        (1,'start','2021-01-01',100,'A'),
        (1,'end','2021-01-31',200,'A'),
        (2,'start','2021-03-05 4:53:11',100,'A'),
        (2,'end','2021-05-01 05:06:38',200,'A'),
]

columns = ["id","session","datetime","station_return","type"]
df = spark.createDataFrame(data=data, schema = columns)
df.createOrReplaceTempView("window_test")
df.show()

+---+-------+-------------------+--------------+----+
| id|session|           datetime|station_return|type|
+---+-------+-------------------+--------------+----+
|  1|  start|         2021-01-01|           100|   A|
|  1|    end|         2021-01-31|           200|   A|
|  2|  start| 2021-03-05 4:53:11|           100|   A|
|  2|    end|2021-05-01 05:06:38|           200|   A|
+---+-------+-------------------+--------------+----+



In [30]:
spark.sql('''
SELECT
  id,
  datetime,
  lead(datetime) OVER (PARTITION BY id ORDER BY datetime) as next_datetime,
  DATEDIFF(lead(datetime) OVER (PARTITION BY id ORDER BY datetime),datetime) as duration_in_days

FROM window_test

''').show()

+---+-------------------+-------------------+----------------+
| id|           datetime|      next_datetime|duration_in_days|
+---+-------------------+-------------------+----------------+
|  1|         2021-01-01|         2021-01-31|              30|
|  1|         2021-01-31|               NULL|            NULL|
|  2| 2021-03-05 4:53:11|2021-05-01 05:06:38|              57|
|  2|2021-05-01 05:06:38|               NULL|            NULL|
+---+-------------------+-------------------+----------------+



## Unpivotting (Despivotando)

In [31]:
from pyspark.sql.types import *


data = [
        ('tim', 10, 9, 8, 5),
        ('john', 5, 6, 3, 6),
        ('jane', 7, 8, 9, 10),

]

schema = StructType([
   StructField("name", StringType(), True),
   StructField("experience", IntegerType(), True),
   StructField("satisfaction", IntegerType(), True),
   StructField("customer_service", IntegerType(), True),
   StructField("speed_of_service", IntegerType(), True)])


df = spark.createDataFrame(data, schema=schema)

df.show()

+----+----------+------------+----------------+----------------+
|name|experience|satisfaction|customer_service|speed_of_service|
+----+----------+------------+----------------+----------------+
| tim|        10|           9|               8|               5|
|john|         5|           6|               3|               6|
|jane|         7|           8|               9|              10|
+----+----------+------------+----------------+----------------+



In [32]:
cols = ['experience', 'satisfaction', 'customer_service', 'speed_of_service']

exprs = f"""stack({len(cols)}, {", ".join([f"'{i}',{i}" for i in cols])}) as (question,score)"""

unpivotted_df = df.select("name",F.expr(exprs))

unpivotted_df.show()

+----+----------------+-----+
|name|        question|score|
+----+----------------+-----+
| tim|      experience|   10|
| tim|    satisfaction|    9|
| tim|customer_service|    8|
| tim|speed_of_service|    5|
|john|      experience|    5|
|john|    satisfaction|    6|
|john|customer_service|    3|
|john|speed_of_service|    6|
|jane|      experience|    7|
|jane|    satisfaction|    8|
|jane|customer_service|    9|
|jane|speed_of_service|   10|
+----+----------------+-----+



## Crear un rango de fechas

In [33]:
date_range_df = spark.sql("SELECT explode(sequence(to_date('2018-01-01'), to_date('2018-03-01'), interval 1 day)) as date")
date_range_df.show()

+----------+
|      date|
+----------+
|2018-01-01|
|2018-01-02|
|2018-01-03|
|2018-01-04|
|2018-01-05|
|2018-01-06|
|2018-01-07|
|2018-01-08|
|2018-01-09|
|2018-01-10|
|2018-01-11|
|2018-01-12|
|2018-01-13|
|2018-01-14|
|2018-01-15|
|2018-01-16|
|2018-01-17|
|2018-01-18|
|2018-01-19|
|2018-01-20|
+----------+
only showing top 20 rows



## Concatenar valores de fila después de la agrupación

In [34]:
df = (spark
    .createDataFrame([
        (1, 'hello',3),
        (2, 'hello',5),
        (3, 'hello',5),
        (3, 'hello',5),
        (3, 'hello',5),
        ],
        ["id", "text"]))

df.createOrReplaceTempView("group_array")

df.show()

+---+-----+---+
| id| text| _3|
+---+-----+---+
|  1|hello|  3|
|  2|hello|  5|
|  3|hello|  5|
|  3|hello|  5|
|  3|hello|  5|
+---+-----+---+



In [35]:
#Devolver cada elemento
spark.sql("Select g.text, collect_list(g.id) FROM group_array as g GROUP BY 1").show()

+-----+----------------+
| text|collect_list(id)|
+-----+----------------+
|hello| [1, 2, 3, 3, 3]|
+-----+----------------+



In [36]:
#Devolver lista única
spark.sql("Select g.text, collect_set(g.id) FROM group_array as g GROUP BY 1").show()

+-----+---------------+
| text|collect_set(id)|
+-----+---------------+
|hello|      [1, 2, 3]|
+-----+---------------+



## Manejo de valores nulos

In [37]:
df = (spark
    .createDataFrame([
        (1, 'hello',None),
        (2, 'hello',None),
        (3, 'hello',5),
        (3, 'hello',5),
        (3, 'hello',5),
        ],
        ["id", "text"]))

df.createOrReplaceTempView("group_array")

df.show()

+---+-----+----+
| id| text|  _3|
+---+-----+----+
|  1|hello|NULL|
|  2|hello|NULL|
|  3|hello|   5|
|  3|hello|   5|
|  3|hello|   5|
+---+-----+----+



In [38]:
spark.sql("Select * from group_array where _3 IS NOT NULL").show()

+---+-----+---+
| id| text| _3|
+---+-----+---+
|  3|hello|  5|
|  3|hello|  5|
|  3|hello|  5|
+---+-----+---+

